## Writing Functions

The goal of this exercise is to replicate the process from the slides, but with multivariate data. You should think of this as a practise of two things: 

1. Performing linear algebra in R. This is straight forward, but takes some practise. Especially worth noting how to use vectors and scalars together, as you see how to modify the functions we originally wrote for scalars, such that they work for vectors.

2. Writing small functions and testing them. Make sure each part works before trying to combine them into a whole!

In [ ]:
library(tidyverse)

In [ ]:
# Write a function that generates data for regressions, 

# y should be generated according to: 
# y <- X %*% beta + eps
# X should be several columns of independent random normals (aka a multivariate random 
# normal matrix with diagonal covariance)
# eps should be such that:
# eps <- rnorm(1, 0, sd)

# beta should be a vector
# params should be a DATAFRAME/TIBBLE with two columns: "mean" and "sd" to generate the features (X) 
# sd should be a scalar: the standard deviation of the normally distributed error term in the DGP

# hint: use cbind to bind vectors into the columns of a matrix or the library MASS with the function mvrnorm

library(MASS)

generate_data <- function(N, beta, params, sd) { 
  
  # Generates independent variables X, with means mu and sd sigma
  X <- mvrnorm(N,params$mu,diag(params$sigma))
  
  # Generates error terms with mean zero (important for OLS) and sd sigma
  eps <- rnorm(N,0,sd)
  
  # Generate data y according to the data generating function 
  y <- X%*% beta + eps
  
  list(X=X, y=y)
}

#### Estimate Coefficients ####

calc_coef <- function(y,x){
  coef <- solve( t(x) %*% x, t(x) %*% y)
  coef
}

#### Standard Errors ####

# Standard Errors 
calc_se <- function(y,x, coef){
  residuals <- y - x %*% coef
  mse <- mean(residuals^2)
  cov_matrix <- mse * solve(t(x)%*%x)
  se <- sqrt(diag(cov_matrix))
  se
}

##### Run Regression ####

run_regression <- function(y,x) {
  coeffs <- calc_coef(y,x)
  se <- calc_se(y,x,calc_coef(y,x))
  list(beta_hat = coeffs, standard_errors = se)
}

##### Evaluate Model ####
eval_model <- function(coef, se, beta, conf = 1.96){
  up <- coef + se * conf
  down <- coef - se*conf
  beta > down & beta < up
}

#### Simulate ####

simulate <- function(N,beta,params,sd){
  d <- generate_data(N, beta, params, sd)
  m <- run_regression(d$y, d$X)
  eval_model(m$beta_hat,m$standard_errors,beta)
}

#### Avg Simulate ####

avg_simulations <- function(M,N,beta,params,sd){
  inside <- sapply(1:M, function(x) {
    simulate(N,beta,params,sd)
  })
  sum(inside) /M
}

In [326]:
# Write other helper functions, as in the slides, and eventually a function "avg_simulations" that takes
# the same parameters as generate_data, plus an "M" parameter that controls how many simulations are run

## Tests

Make sure the following assertions pass:

In [ ]:
a <- avg_simulations(1000, 
                20, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(round(a, 1) == 1.9)

In [ ]:
a <- avg_simulations(1000, 
                500, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(a > .92)